In [30]:
import pandas as pd 
from sklearn.pipeline import Pipeline
from datacleaning.FetchData import FetchData
from datacleaning.CleanData import CleanData
from datacleaning.pipelineclasses import *

In [31]:
FetchData.scan_save_all_records()
CleanData.clean_save_raw_data()

IndexError: list index out of range

In [17]:
raw = pd.read_csv("data/raw_data.csv")
raw.head()

,Unnamed: 0,vehicle_maxChgRate_W,peakPower_W,sch_centsPerHr,connectTime,user_email,vehicle_model,Duration,userId,regular,...,DurationHrs,dcosId,lastUpdate,energyReq_Wh,power,stationId,defaultDeadline,scheduled,cumEnergy_Wh,reg_centsPerHr
0,0,8800,3380,150.0,2022-01-07T06:23:52,lamartell@berkeley.edu,Prius Prime,0 days 01:40:50,603,0,...,1.68055,1838,2022-01-07T08:05:09,8800.0,"[{'power_W': Decimal('1229'), 'timestamp': Dec...",12,2022-01-07T08:15:00,1,4214,200.0
1,1,6600,6563,250.0,2022-10-20T10:16:13,erfanm@berkeley.edu,Clarity PHEV,0 days 01:53:35,793,1,...,1.89305,3416,2022-10-20T12:10:08,NaN,"[{'power_W': Decimal('6563'), 'timestamp': Dec...",17,2022-10-20T15:30:00,0,11724,137.0
2,2,3600,3442,142.0,2022-03-22T15:39:06,nthaman@berkeley.edu,Volt,0 days 04:26:19,804,1,...,4.43861,2289,2022-03-23T14:30:09,NaN,"[{'power_W': Decimal('0'), 'timestamp': Decima...",16,2022-03-23T15:45:00,0,14529,129.0
3,3,170000,5170,250.0,2022-06-24T09:57:06,scott.moura@gmail.com,Model 3,0 days 06:37:26,615,1,...,6.62388,2733,2022-06-24T16:35:07,NaN,"[{'power_W': Decimal('0'), 'timestamp': Decima...",16,2022-06-24T21:30:00,0,34904,137.0
4,4,19200,6942,100.0,2021-09-28T09:47:05,lburbidg@ucsd.edu,Model Y,0 days 03:52:57,897,1,...,3.88249,1007,2021-09-28T13:40:07,NaN,"[{'power_W': Decimal('6784'), 'timestamp': Dec...",4,2021-09-28T18:15:00,0,25385,75.0


In [ ]:
pipe = Pipeline(
    [
        ("1", 